# About

Convert ssd_mobilenet_v2_coco_2018_03_29 to Core ML

The code is based on this: https://github.com/hollance/coreml-survival-guide/blob/master/MobileNetV2%2BSSDLite/ssdlite.py



Instead of using tfcoreml to convert from TF to Core ML. We use coremltools for conversion. The model is converted successful, but predict too many bounding boxes.


# Download model and setup environment

In [1]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
!tar -zxvf ssd_mobilenet_v2_coco_2018_03_29.tar.gz
!wget https://raw.githubusercontent.com/hollance/coreml-survival-guide/master/MobileNetV2%2BSSDLite/coco_labels.txt

--2021-08-12 07:43:13--  http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.152.128, 2607:f8b0:4001:c56::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.152.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 187925923 (179M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_coco_2018_03_29.tar.gz’

ssd_mobilenet_v2_co 100%[===================>] 179.22M   243MB/s    in 0.7s    

2021-08-12 07:43:14 (243 MB/s) - ‘ssd_mobilenet_v2_coco_2018_03_29.tar.gz’ saved [187925923/187925923]

ssd_mobilenet_v2_coco_2018_03_29/checkpoint
ssd_mobilenet_v2_coco_2018_03_29/model.ckpt.meta
ssd_mobilenet_v2_coco_2018_03_29/pipeline.config
ssd_mobilenet_v2_coco_2018_03_29/saved_model/saved_model.pb
ssd_mobilenet_v2_coco_2018_03_29/frozen_inference_graph.pb
ssd_mobilenet_v2_coco_2018_03_29/saved_model/
ssd_mobilenet_v2_coco_2018_03_29/saved_mo

In [2]:
!pip install coremltools==4.1

     |████████████████████████████████| 3.4 MB 5.3 MB/s 
  Created wheel for attr: filename=attr-0.3.1-py3-none-any.whl size=2457 sha256=cc312b020c2984c41acf4313469d8d6fe9de312846562b28ed07612cc938e1e5
  Stored in directory: /root/.cache/pip/wheels/3b/5d/58/41fbe92f47031641008bd8559ee89e58bf0f123f9c18dea1cb
Successfully built attr


In [1]:
%tensorflow_version 1.x

import sys
print(sys.version)

import tensorflow as tf
print(tf.__version__)

import coremltools as ct
print(ct.__version__)

TensorFlow 1.x selected.
3.7.11 (default, Jul  3 2021, 18:01:19) 
[GCC 7.5.0]
1.15.2


4.1


# Convert to Core ML

In [10]:
import numpy as np

import tensorflow as tf
from tensorflow.python.tools import strip_unused_lib
from tensorflow.python.framework import dtypes
from tensorflow.python.platform import gfile


# From where to load the saved_model.pb file.
saved_model_path = "ssd_mobilenet_v2_coco_2018_03_29/saved_model"

# Where to save the final Core ML model file.
coreml_model_path = "SSDMobileNetV2.mlmodel"

# The number of predicted classes, excluding background.
num_classes = 90

# The number of predicted bounding boxes.
num_anchors = 1917

num_coordinates = 4

# Size of the expected input image.
input_width = 300
input_height = 300


# Temporary file. You can delete this after the conversion is done.
frozen_model_file = "frozen_model.pb"

# Names of the interesting tensors in the graph. We use "Postprocessor/convert_scores"
# instead of "concat_1" because this already applies the sigmoid to the class scores.
input_name_image = "Preprocessor/sub"
output_name_box = "concat"
output_name_score = "Postprocessor/convert_scores"


def load_saved_model(path):
    """Loads a saved model into a graph."""
    print("Loading saved_model.pb from '%s'" % path)
    the_graph = tf.Graph()
    with tf.Session(graph=the_graph) as sess:
        tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], path)
    return the_graph


def optimize_graph(graph):
    """Strips unused subgraphs and save it as another frozen TF model."""
    gdef = strip_unused_lib.strip_unused(
            input_graph_def = graph.as_graph_def(),
            input_node_names = [input_node],
            output_node_names = [bbox_output_node, class_output_node],
            placeholder_type_enum = dtypes.float32.as_datatype_enum)

    with gfile.GFile(frozen_model_file, "wb") as f:
        f.write(gdef.SerializeToString())


# Load the original graph and remove anything we don't need.
the_graph = load_saved_model(saved_model_path)
optimize_graph(the_graph)

Loading saved_model.pb from 'ssd_mobilenet_v2_coco_2018_03_29/saved_model'


In [17]:
# ================================
# PART 1: Convert the SSD to Core ML
# ================================

image_input = ct.ImageType(name=input_name_image,
                           shape=(1, input_width, input_height, 3),
                           bias=[-1,-1,-1], scale=2./255)

ssd_model = ct.convert(
    frozen_model_file,
    source='tensorflow',
    inputs=[image_input],
    outputs=[output_name_score, output_name_box]  # the order in list is important
)

print(ssd_model)

Translating MIL ==> MLModel Ops: 100%|██████████| 1052/1052 [00:02<00:00, 409.19 ops/s]


input {
  name: "Preprocessor/sub"
  type {
    imageType {
      width: 300
      height: 300
      colorSpace: RGB
    }
  }
}
output {
  name: "Postprocessor/convert_scores"
  type {
    multiArrayType {
      dataType: FLOAT32
    }
  }
}
output {
  name: "concat"
  type {
    multiArrayType {
      dataType: FLOAT32
    }
  }
}
metadata {
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "tensorflow==1.15.2"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "4.1"
  }
}



In [18]:
# add missing shape information

ssd_model.input_description[input_name_image] = "Input image to be detected"
ssd_model.output_description[output_name_score] = "Confidence derived for each of the bounding boxes."
ssd_model.output_description[output_name_box] = "Bounding boxes coordinates"

ssd_spec = ssd_model.get_spec()
ct.utils.rename_feature(ssd_spec, input_name_image, "image")
ct.utils.rename_feature(ssd_spec, output_name_score, "scores")
ct.utils.rename_feature(ssd_spec, output_name_box, "boxes")

# scores
ssd_spec.description.output[0].type.multiArrayType.shape.append(num_classes + 1)
ssd_spec.description.output[0].type.multiArrayType.shape.append(num_anchors)

# boxes
ssd_spec.description.output[1].type.multiArrayType.shape.append(num_coordinates)
ssd_spec.description.output[1].type.multiArrayType.shape.append(num_anchors)

ssd_spec.description.output[0].type.multiArrayType.dataType = ct.proto.FeatureTypes_pb2.ArrayFeatureType.DOUBLE
ssd_spec.description.output[1].type.multiArrayType.dataType = ct.proto.FeatureTypes_pb2.ArrayFeatureType.DOUBLE


ssd_model = ct.models.MLModel(ssd_spec)
ssd_model.save("SSD.mlmodel")

print(ssd_model)

input {
  name: "image"
  shortDescription: "Input image to be detected"
  type {
    imageType {
      width: 300
      height: 300
      colorSpace: RGB
    }
  }
}
output {
  name: "scores"
  shortDescription: "Confidence derived for each of the bounding boxes."
  type {
    multiArrayType {
      shape: 91
      shape: 1917
      dataType: DOUBLE
    }
  }
}
output {
  name: "boxes"
  shortDescription: "Bounding boxes coordinates"
  type {
    multiArrayType {
      shape: 4
      shape: 1917
      dataType: DOUBLE
    }
  }
}
metadata {
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "tensorflow==1.15.2"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "4.1"
  }
}



In [19]:
# ================================
# PART 2: Decoding the coordinates
# ================================

def get_anchors(graph, tensor_name):
    """
    Computes the list of anchor boxes by sending a fake image through the graph.
    Outputs an array of size (4, num_anchors) where each element is an anchor box
    given as [ycenter, xcenter, height, width] in normalized coordinates.
    """
    image_tensor = graph.get_tensor_by_name("image_tensor:0")
    box_corners_tensor = graph.get_tensor_by_name(tensor_name)
    box_corners = sess.run(box_corners_tensor, feed_dict={image_tensor: np.zeros((1, input_height, input_width, 3))})

    # The TensorFlow graph gives each anchor box as [ymin, xmin, ymax, xmax]. 
    # Convert these min/max values to a center coordinate, width and height.
    ymin, xmin, ymax, xmax = np.transpose(box_corners)
    width = xmax - xmin
    height = ymax - ymin
    ycenter = ymin + height / 2.
    xcenter = xmin + width / 2.
    return np.stack([ycenter, xcenter, height, width])


# Read the anchors into a (4, 1917) tensor.
anchors_tensor = "Concatenate/concat:0"
with the_graph.as_default():
    with tf.Session(graph=the_graph) as sess:
        anchors = get_anchors(the_graph, anchors_tensor)
        assert(anchors.shape[1] == num_anchors)


from coremltools.models import datatypes
from coremltools.models import neural_network

# MLMultiArray inputs of neural networks must have 1 or 3 dimensions. 
# We only have 2, so add an unused dimension of size one at the back.
input_features = [ ("scores", datatypes.Array(num_classes + 1, num_anchors, 1)),
                   ("boxes", datatypes.Array(4, num_anchors, 1)) ]

# The outputs of the decoder model should match the inputs of the next
# model in the pipeline, NonMaximumSuppression. This expects the number
# of bounding boxes in the first dimension.
output_features = [ ("raw_confidence", datatypes.Array(num_anchors, num_classes)),
                    ("raw_coordinates", datatypes.Array(num_anchors, 4)) ]

builder = neural_network.NeuralNetworkBuilder(input_features, output_features)

# (num_classes+1, num_anchors, 1) --> (1, num_anchors, num_classes+1)
builder.add_permute(name="permute_scores",
                    dim=(0, 3, 2, 1),
                    input_name="scores",
                    output_name="permute_scores_output")

# Strip off the "unknown" class (at index 0).
builder.add_slice(name="slice_scores",
                  input_name="permute_scores_output",
                  output_name="raw_confidence",
                  axis="width",
                  start_index=1,
                  end_index=num_classes + 1)

# Grab the y, x coordinates (channels 0-1).
builder.add_slice(name="slice_yx",
                  input_name="boxes",
                  output_name="slice_yx_output",
                  axis="channel",
                  start_index=0,
                  end_index=2)

# boxes_yx / 10
builder.add_elementwise(name="scale_yx",
                        input_names="slice_yx_output",
                        output_name="scale_yx_output",
                        mode="MULTIPLY",
                        alpha=0.1)

# Split the anchors into two (2, 1917, 1) arrays.
anchors_yx = np.expand_dims(anchors[:2, :], axis=-1)
anchors_hw = np.expand_dims(anchors[2:, :], axis=-1)

builder.add_load_constant(name="anchors_yx",
                          output_name="anchors_yx",
                          constant_value=anchors_yx,
                          shape=[2, num_anchors, 1])

builder.add_load_constant(name="anchors_hw",
                          output_name="anchors_hw",
                          constant_value=anchors_hw,
                          shape=[2, num_anchors, 1])

# (boxes_yx / 10) * anchors_hw
builder.add_elementwise(name="yw_times_hw",
                        input_names=["scale_yx_output", "anchors_hw"],
                        output_name="yw_times_hw_output",
                        mode="MULTIPLY")

# (boxes_yx / 10) * anchors_hw + anchors_yx
builder.add_elementwise(name="decoded_yx",
                        input_names=["yw_times_hw_output", "anchors_yx"],
                        output_name="decoded_yx_output",
                        mode="ADD")

# Grab the height and width (channels 2-3).
builder.add_slice(name="slice_hw",
                  input_name="boxes",
                  output_name="slice_hw_output",
                  axis="channel",
                  start_index=2,
                  end_index=4)

# (boxes_hw / 5)
builder.add_elementwise(name="scale_hw",
                        input_names="slice_hw_output",
                        output_name="scale_hw_output",
                        mode="MULTIPLY",
                        alpha=0.2)

# exp(boxes_hw / 5)
builder.add_unary(name="exp_hw",
                  input_name="scale_hw_output",
                  output_name="exp_hw_output",
                  mode="exp")

# exp(boxes_hw / 5) * anchors_hw
builder.add_elementwise(name="decoded_hw",
                        input_names=["exp_hw_output", "anchors_hw"],
                        output_name="decoded_hw_output",
                        mode="MULTIPLY")

# The coordinates are now (y, x) and (height, width) but NonMaximumSuppression
# wants them as (x, y, width, height). So create four slices and then concat
# them into the right order.
builder.add_slice(name="slice_y",
                  input_name="decoded_yx_output",
                  output_name="slice_y_output",
                  axis="channel",
                  start_index=0,
                  end_index=1)

builder.add_slice(name="slice_x",
                  input_name="decoded_yx_output",
                  output_name="slice_x_output",
                  axis="channel",
                  start_index=1,
                  end_index=2)

builder.add_slice(name="slice_h",
                  input_name="decoded_hw_output",
                  output_name="slice_h_output",
                  axis="channel",
                  start_index=0,
                  end_index=1)

builder.add_slice(name="slice_w",
                  input_name="decoded_hw_output",
                  output_name="slice_w_output",
                  axis="channel",
                  start_index=1,
                  end_index=2)

builder.add_elementwise(name="concat",
                        input_names=["slice_x_output", "slice_y_output", 
                                     "slice_w_output", "slice_h_output"],
                        output_name="concat_output",
                        mode="CONCAT")

# (4, num_anchors, 1) --> (1, num_anchors, 4)
builder.add_permute(name="permute_output",
                    dim=(0, 3, 2, 1),
                    input_name="concat_output",
                    output_name="raw_coordinates")

decoder_model = ct.models.MLModel(builder.spec)
decoder_model.save("Decoder.mlmodel")

print(decoder_model)

input {
  name: "scores"
  type {
    multiArrayType {
      shape: 91
      shape: 1917
      shape: 1
      dataType: DOUBLE
    }
  }
}
input {
  name: "boxes"
  type {
    multiArrayType {
      shape: 4
      shape: 1917
      shape: 1
      dataType: DOUBLE
    }
  }
}
output {
  name: "raw_confidence"
  type {
    multiArrayType {
      shape: 1917
      shape: 90
      dataType: DOUBLE
    }
  }
}
output {
  name: "raw_coordinates"
  type {
    multiArrayType {
      shape: 1917
      shape: 4
      dataType: DOUBLE
    }
  }
}



In [20]:
# ===============================
# PART 3: Non-maximum suppression
# ===============================

nms_spec = ct.proto.Model_pb2.Model()
nms_spec.specificationVersion = 3

for i in range(2):
    decoder_output = decoder_model._spec.description.output[i].SerializeToString()

    nms_spec.description.input.add()
    nms_spec.description.input[i].ParseFromString(decoder_output)

    nms_spec.description.output.add()
    nms_spec.description.output[i].ParseFromString(decoder_output)
    
nms_spec.description.output[0].name = "confidence"
nms_spec.description.output[1].name = "coordinates"

output_sizes = [num_classes, 4]
for i in range(2):
    ma_type = nms_spec.description.output[i].type.multiArrayType
    ma_type.shapeRange.sizeRanges.add()
    ma_type.shapeRange.sizeRanges[0].lowerBound = 0
    ma_type.shapeRange.sizeRanges[0].upperBound = -1
    ma_type.shapeRange.sizeRanges.add()
    ma_type.shapeRange.sizeRanges[1].lowerBound = output_sizes[i]
    ma_type.shapeRange.sizeRanges[1].upperBound = output_sizes[i]
    del ma_type.shape[:]

nms = nms_spec.nonMaximumSuppression
nms.confidenceInputFeatureName = "raw_confidence"
nms.coordinatesInputFeatureName = "raw_coordinates"
nms.confidenceOutputFeatureName = "confidence"
nms.coordinatesOutputFeatureName = "coordinates"
nms.iouThresholdInputFeatureName = "iouThreshold"
nms.confidenceThresholdInputFeatureName = "confidenceThreshold"

default_iou_threshold = 0.6
default_confidence_threshold = 0.4
nms.iouThreshold = default_iou_threshold
nms.confidenceThreshold = default_confidence_threshold

nms.pickTop.perClass = True

labels = np.loadtxt("coco_labels.txt", dtype=str, delimiter="\n")
nms.stringClassLabels.vector.extend(labels)

nms_model = ct.models.MLModel(nms_spec)
nms_model.save("NMS.mlmodel")

print(nms_model)

input {
  name: "raw_confidence"
  type {
    multiArrayType {
      shape: 1917
      shape: 90
      dataType: DOUBLE
    }
  }
}
input {
  name: "raw_coordinates"
  type {
    multiArrayType {
      shape: 1917
      shape: 4
      dataType: DOUBLE
    }
  }
}
output {
  name: "confidence"
  type {
    multiArrayType {
      dataType: DOUBLE
      shapeRange {
        sizeRanges {
          upperBound: -1
        }
        sizeRanges {
          lowerBound: 90
          upperBound: 90
        }
      }
    }
  }
}
output {
  name: "coordinates"
  type {
    multiArrayType {
      dataType: DOUBLE
      shapeRange {
        sizeRanges {
          upperBound: -1
        }
        sizeRanges {
          lowerBound: 4
          upperBound: 4
        }
      }
    }
  }
}



In [21]:
# ===============================================
# PART 4: Putting it all together into a pipeline
# ===============================================

from coremltools.models.pipeline import *

input_features = [ ("image", datatypes.Array(3, 300, 300)),
                   ("iouThreshold", datatypes.Double()),
                   ("confidenceThreshold", datatypes.Double()) ]

output_features = [ "confidence", "coordinates" ]

pipeline = Pipeline(input_features, output_features)

# We added a dimension of size 1 to the back of the inputs of the decoder 
# model, so we should also add this to the output of the SSD model or else 
# the inputs and outputs do not match and the pipeline is not valid.
ssd_output = ssd_model._spec.description.output
ssd_output[0].type.multiArrayType.shape[:] = [num_classes + 1, num_anchors, 1]
ssd_output[1].type.multiArrayType.shape[:] = [4, num_anchors, 1]

pipeline.add_model(ssd_model)
pipeline.add_model(decoder_model)
pipeline.add_model(nms_model)

# The "image" input should really be an image, not a multi-array.
pipeline.spec.description.input[0].ParseFromString(ssd_model._spec.description.input[0].SerializeToString())

# Copy the declarations of the "confidence" and "coordinates" outputs.
# The Pipeline makes these strings by default.
pipeline.spec.description.output[0].ParseFromString(nms_model._spec.description.output[0].SerializeToString())
pipeline.spec.description.output[1].ParseFromString(nms_model._spec.description.output[1].SerializeToString())

# Add descriptions to the inputs and outputs.
pipeline.spec.description.input[1].shortDescription = f"(optional) IOU Threshold override (default={default_iou_threshold}"
pipeline.spec.description.input[2].shortDescription = f"(optional) Confidence Threshold override (default={default_confidence_threshold})"
pipeline.spec.description.output[0].shortDescription = u"Boxes \xd7 Class confidence"
pipeline.spec.description.output[1].shortDescription = u"Boxes \xd7 [x, y, width, height] (relative to image size)"

# Add metadata to the model.
pipeline.spec.description.metadata.versionString = "ssd_mobilenet_v2_coco_2018_03_29"
pipeline.spec.description.metadata.shortDescription = """
MobileNetV2 + SSD, trained on COCO. Source: http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
"""
pipeline.spec.description.metadata.author = "Converted to Core ML by Anh"
pipeline.spec.description.metadata.license = "https://github.com/tensorflow/models/blob/master/research/object_detection"

# Add the list of class labels and the default threshold values too.
user_defined_metadata = {
    "iou_threshold": str(default_iou_threshold),
    "confidence_threshold": str(default_confidence_threshold),
    "classes": ",".join(labels)
}
pipeline.spec.description.metadata.userDefined.update(user_defined_metadata)

# Don't forget this or Core ML might attempt to run the model on an unsupported
# operating system version!
pipeline.spec.specificationVersion = 3

final_model = ct.models.MLModel(pipeline.spec)
final_model.save(coreml_model_path)

print(final_model)
print("Done!")

input {
  name: "image"
  shortDescription: "Input image to be detected"
  type {
    imageType {
      width: 300
      height: 300
      colorSpace: RGB
    }
  }
}
input {
  name: "iouThreshold"
  shortDescription: "(optional) IOU Threshold override (default=0.6"
  type {
    doubleType {
    }
  }
}
input {
  name: "confidenceThreshold"
  shortDescription: "(optional) Confidence Threshold override (default=0.4)"
  type {
    doubleType {
    }
  }
}
output {
  name: "confidence"
  shortDescription: "Boxes \303\227 Class confidence"
  type {
    multiArrayType {
      dataType: DOUBLE
      shapeRange {
        sizeRanges {
          upperBound: -1
        }
        sizeRanges {
          lowerBound: 90
          upperBound: 90
        }
      }
    }
  }
}
output {
  name: "coordinates"
  shortDescription: "Boxes \303\227 [x, y, width, height] (relative to image size)"
  type {
    multiArrayType {
      dataType: DOUBLE
      shapeRange {
        sizeRanges {
          upperBound: